In [11]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import torch
import pandas as pd
import numpy as np
import cv2
import os
from PIL import Image, ImageFile
from tqdm import tqdm

device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True


INPUT_ROOT = r'../input'

MODEL_ROOT = r'../input/models'
MODEL_NAME = r'mobilenet_00'
MODEL_PATH = os.path.join(MODEL_ROOT, MODEL_NAME)


EVALUATION_PERCENTAGE = 0.1

# TODO figure out if large images work
IMG_SIZE = 512

NUM_CLASSES = 5

BATCH_SIZE = 16
NUM_WORKERS = 8

In [12]:
def transform_ndarray2tensor():
    return transforms.Compose([
        transforms.ToTensor(),
        # normalize the images to torchvision models specifications
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])


def crop_image_from_gray(img, tol=7):
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol

        check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if (check_shape == 0):  # image is too dark so that we crop out everything,
            return img  # return original image
        else:
            img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
            img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
            img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
            #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1, img2, img3], axis=-1)
        #         print(img.shape)
        return img

def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), sigmaX), -4, 128)

    return image

In [13]:
class BlindnessTestDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.transform = transform_ndarray2tensor()
        self.col_id = self.data.columns.get_loc('id_code')      # should be 0

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        img_name = os.path.join(INPUT_ROOT, 'test_images', self.data.iat[index, self.col_id] + '.png')
        image = load_ben_color(img_name)
        image = self.transform(image)
        return image


In [15]:
test_df = pd.read_csv(os.path.join(INPUT_ROOT, 'test.csv'))
test_ds = BlindnessTestDataset(test_df)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

model = torch.load(MODEL_PATH)
model.to(device)
predictions_list = []
with tqdm(range(len(test_dl))) as pbar:
    for inputs in test_dl:
        inputs = inputs.to(device)
        output = model(inputs)
        predictions = torch.argmax(output, dim=-1)
        predictions_list.append(predictions.cpu().numpy())
        pbar.update()
all_predictions = np.concatenate(predictions_list)
print(all_predictions.shape)

  0%|          | 0/229 [00:00<?, ?it/s]


BrokenPipeError: [Errno 32] Broken pipe

In [10]:
test_df['diagnosis'] = all_predictions
test_df.to_csv('submission.csv', index=False)

NameError: name 'all_predictions' is not defined